In [5]:

import xml.etree.ElementTree as ET
import os
import json

# path=(r"C:\Project\TAC\train_xml\APTIOM.xml")
path=(r"C:\Project\TAC\train_xml")
# Parsing the XML
for filename in os.listdir(path):
    # print(filename)
    if not filename.endswith('.xml'): continue
    fullname = os.path.join(path, filename)
    # print(fullname)
    tree = ET.parse(fullname)
    root = tree.getroot()
    # print(root)

    # Extracting the text contents
    json_arr=[]
    json_obj={}
    content=""
    for text in root.findall("Text/Section"):
    #   content+=text.text.split("\n\n")
        content+=text.text

    # extracting mention tag
    mentions = []

    for mention_val in root.iter('Mention'):
        mention_val.attrib["String"]=mention_val.attrib.pop("str")
        mention_val.set('end',int(0))
        mention = mention_val.attrib

        mention['end']=mention['start'].split(",")[-1]
        mention['start']=mention['start'].split(",")[0]
        # print(mention['start'])
        # mention[]
        mention["len"]=mention['len'].split(",")[-1]

        if mention and mention.get("section") == "S2":
            mention['start'] = int(mention['start'])+len(root.findall('Text/Section')[0].text)
            mention['end'] = int(mention['end'])+int(mention['len'])+len(root.findall('Text/Section')[0].text)
        elif mention.get("section") == "S3":
                mention['start'] = int(mention['start'])+len(root.findall('Text/Section')[0].text)+len(root.findall('Text/Section')[1].text)
                mention['end'] = int(mention['end'])+int(mention['len'])+len(root.findall('Text/Section')[0].text)+len(root.findall('Text/Section')[1].text)                                                                                     
        else:
            mention['end'] = int(mention['end'])+int(mention['len'])
            
        mentions.append(mention)
        # print(mentions)
    relations=[]
    for relation_val in root.iter('Relation'):
        relations.append(relation_val.attrib)
    #   print(relation)

    json_obj["corpus"]="TAC"
    json_obj["metadata"]="train"
    json_obj["id"]=filename.split(".")[0]
    json_obj["Text"]=content
    json_obj["annotation"]=mentions
    json_obj["relation"]=relations
    json_arr.append(json_obj)
    json_filename = filename.replace('xml', 'json')
    with open(os.path.join(r"C:\Project\TAC\train_json", json_filename), "w") as f:
        json.dump(json_arr, f)

  